Recorded ping tests: 1 per minute, single ping, to 8.8.8.8 (google dns)

Data quality notes: At one point in time we had a timezone jump because the timezone was corrected on the measuring device. Timezone was recorded, but I chose to keep processing simple and ignore timezones as much as possible

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataset
import datetime
from collections import Counter

In [8]:
ds = dataset.load_dataset('googledns.log')
len(ds)

591341

In [11]:
success_count = sum(map(lambda datapoint: datapoint[1], ds))
fail_count = len(list(filter(lambda datapoint: datapoint[1] == 0, ds)))
print("Successful:", success_count)
print("Failures  :", fail_count)
print("Total     :", (success_count + fail_count))

Successful: 587217
Failures  : 4124
Total     : 591341


In [32]:
# rough estimate of the number of hours of not being able to ping google
print("Estimated hour of failure:", (fail_count / 60))

Estimated hour of failure: 68.73333333333333


In [62]:
# Let's find when things went wrong

# find points at which is_good(previous) XOR is_good(current)
changepoints = [(0, ds[0][0], 0)]
for i in range(len(ds) - 1):
    data, received = ds[i]
    if received ^ ds[i+1][1]:
        changepoints.append((i+1, ds[i+1][0], ds[i+1][1])) #idx, timestamp, status

In [63]:
outages = []
for i in range(len(changepoints)):
    idx, start, status = changepoints[i]
    if status == 0:
        if i+1 < len(changepoints):
            end = changepoints[i+1][1]
            next_idx = changepoints[i+1][0]
        else:
            end = datetime.datetime()
            next_idx = -1
        duration = (end - start)
        outages.append((start, duration, next_idx - idx))

# disca single ping failures.

In [75]:
for outage in outages:
    start, duration, samples = outage

    print("No successful pings to 8.8.8.8 from %s: %s (# failed: %s)"%(start, duration, samples))

No successful pings to 8.8.8.8 from 2020-06-25 06:54:34: 11:20:42 (# failed: 630)
No successful pings to 8.8.8.8 from 2020-06-25 18:15:16: 0:19:45 (# failed: 21)
No successful pings to 8.8.8.8 from 2020-07-02 16:09:01: 0:01:00 (# failed: 1)
No successful pings to 8.8.8.8 from 2020-07-28 00:18:01: 0:05:00 (# failed: 5)
No successful pings to 8.8.8.8 from 2020-07-28 00:39:01: 0:03:01 (# failed: 3)
No successful pings to 8.8.8.8 from 2020-07-31 04:03:01: 0:02:00 (# failed: 2)
No successful pings to 8.8.8.8 from 2020-07-31 13:00:01: 0:10:00 (# failed: 10)
No successful pings to 8.8.8.8 from 2020-07-31 14:01:01: 0:29:00 (# failed: 29)
No successful pings to 8.8.8.8 from 2020-08-01 05:02:01: 6:38:00 (# failed: 397)
No successful pings to 8.8.8.8 from 2020-08-01 14:58:01: 0:14:00 (# failed: 14)
No successful pings to 8.8.8.8 from 2020-08-07 13:20:01: 0:05:00 (# failed: 5)
No successful pings to 8.8.8.8 from 2020-08-07 13:37:01: 0:02:00 (# failed: 2)
No successful pings to 8.8.8.8 from 2020-08

In [114]:
outage_counter = {}

for outage in outages:
    start, duration, samples = outage
    
    if samples < 2: continue

    year_month = "%d-%02d"%(start.year, start.month)
    updated_total = outage_counter.setdefault(year_month, 0) + samples
    outage_counter[year_month]=updated_total
    
for ym in sorted(outage_counter.keys()):
    pingfails = outage_counter[ym]
    print("Total missed pings to 8.8.8.8 on %s: %5d (%.2f hours of degraded connectivity)"%(ym, pingfails, pingfails / 60))

Total missed pings to 8.8.8.8 on 2020-06:   651 (10.85 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-07:    49 (0.82 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-08:  2199 (36.65 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-09:     8 (0.13 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-10:     2 (0.03 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-11:     9 (0.15 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2020-12:     7 (0.12 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-01:    98 (1.63 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-02:    18 (0.30 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-03:     9 (0.15 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-04:     8 (0.13 hours of degraded connectivity)
Total missed pings to 8.8.8.8 on 2021-05:    15 (0.2